<a href="https://colab.research.google.com/github/swiftautoai/receipt/blob/main/main_receipt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ติดตั้ง dependencies (เฉพาะครั้งแรก)
!pip install numpy==1.23.5 --quiet
!pip install gspread==5.11.1 oauth2client==4.1.3 --quiet
!pip install google-cloud-vision --quiet

import os, json
import sys
from google.colab import drive


# STEP 1: Mount Google Drive
drive.mount('/content/drive')

# STEP 2: ระบุ path หลักเพียงจุดเดียว
project_drive_path = "/content/drive/MyDrive/swiftautoai/receipt"   # ✅เปลี่ยน working directory
setting_path = os.path.join(project_drive_path, "setting")
notebook_name = "main_receipt.ipynb"

# สร้างโฟลเดอร์ setting หากยังไม่มี
os.makedirs(setting_path, exist_ok=True)

# เปลี่ยน working directory ไปยัง receipt (หรือจะเปลี่ยนเป็น setting ก็ได้ตามจุดประสงค์)
os.chdir(project_drive_path)

# ตรวจว่า notebook ถูกวางไว้ในโฟลเดอร์ setting หรือไม่
expected_notebook_path = os.path.join(setting_path, notebook_name)
if not os.path.exists(expected_notebook_path):
    print(f"\033[91m❌ กรุณาย้ายไฟล์ {notebook_name} ไปไว้ที่:\n📂 {setting_path}\033[0m")
    sys.exit()
else:
    print(f"\033[92m✅ พบไฟล์ {notebook_name} แล้วใน setting พร้อมใช้งานต่อได้เลย\033[0m")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 47.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.8 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
xarray-einstats 0.9.1 require

In [2]:
# STEP 2.1: ตั้งค่า PROJECT_DIR และ path สำหรับ config
PROJECT_DIR = "."  # ใช้ path สัมพัทธ์จาก receipt/
CONFIG_PATH = os.path.join(setting_path, "config.json")

# ถ้ายังไม่มี config.json → สร้างใหม่
if not os.path.exists(CONFIG_PATH):
    config_data = {
        "SPREADSHEET_NAME": "receipt",
        "INPUT_FOLDER": "bills_input",
        "PROCESSED_FOLDER": "bills_processed",
        "SERVICE_ACCOUNT_PATH": "service_account.json",
        "SERVICE_ACCOUNT_FILE": "service_account.json",
        "FOLDER_PATH": "bills_input",

        "API_KEY_GEMINI": "AIzaSyDbC0f8rU4_xxxxxxxxxxxxxxxxxxxxxx",
        "FOLDER_ID": "1ilXA_xxxxxxxxxxxxxxxxx"

    }

    with open(CONFIG_PATH, "w") as f:
        json.dump(config_data, f, indent=2)

    #print("✅ สร้างไฟล์ config.json ใหม่เรียบร้อยแล้ว")

# โหลด config.json (ไม่ว่าเพิ่งสร้างหรือมีอยู่แล้ว)
with open(CONFIG_PATH, "r") as f:
    config = json.load(f)


# STEP 2.2: โหลดค่าจาก config
INPUT_FOLDER = os.path.join(PROJECT_DIR, config["INPUT_FOLDER"])                  #config.json
PROCESSED_FOLDER = os.path.join(PROJECT_DIR, config["PROCESSED_FOLDER"])          #config.json
SERVICE_ACCOUNT_FILE = os.path.join(PROJECT_DIR, "setting", config["SERVICE_ACCOUNT_FILE"])  #config.json


os.makedirs(PROCESSED_FOLDER, exist_ok=True)
os.makedirs(INPUT_FOLDER, exist_ok=True)

# ✅ ตั้งค่าตำแหน่งไฟล์ Service Account สำหรับ Google Vision
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_FILE

In [3]:
# ✅ STEP 2.5: ตรวจชื่อซ้ำและ image hash แบบง่าย ๆ
!pip install ImageHash --quiet
from PIL import Image
import imagehash
import os
from glob import glob
import sys

def get_image_hash(file_path):
    """คำนวณ image hash สำหรับเปรียบเทียบความเหมือน"""
    try:
        with Image.open(file_path) as img:
            return str(imagehash.average_hash(img))
    except Exception as e:
        print(f"❌ Error hashing {file_path}: {e}")
        return None

def show_alert_message(title, message, message_type="warning"):
    """แสดงข้อความแจ้งเตือนแบบเด่นชัด"""
    border = "=" * 80
    if message_type == "error":
        icon = "🚨"
        color_code = "\033[92m"  # Red "\033[91m"
    elif message_type == "warning":
        icon = "⚠️"
        color_code = "\033[92m"  # Yellow "\033[93m"
    else:
        icon = "ℹ️"
        color_code = "\033[92m"  # Blue "\033[94m"

    reset_code = "\033[0m"

    print(f"\n{color_code}{border}")
    print(f"{icon} {title}")
    print(f"{border}")
    print(f"{message}")
    print(f"{border}{reset_code}\n")

# ✅ STEP 3: เตรียมรายการไฟล์ใหม่
image_files = glob(INPUT_FOLDER + '/*.JPG') + glob(INPUT_FOLDER + '/*.jpg') + glob(INPUT_FOLDER + '/*.png')
image_files = sorted(image_files)

# เตรียมเซตเก็บ hash และชื่อไฟล์ที่เคยประมวลผล
existing_hashes = {}
existing_filenames = set()

# ดึงข้อมูลไฟล์ที่เคยประมวลผลจากโฟลเดอร์ PROCESSED
print("🔍 โหลดข้อมูลไฟล์ที่เคยประมวลผล...")
for processed_file in glob(PROCESSED_FOLDER + '/*'):
    filename = os.path.basename(processed_file)
    h = get_image_hash(processed_file)
    if h:
        existing_hashes[filename] = h
        existing_filenames.add(filename)

print(f"📋 พบไฟล์ที่เคยประมวลผล: {len(existing_filenames)} ไฟล์")

# ✅ STEP 4: ตรวจสอบไฟล์ซ้ำและรวบรวมรายการปัญหา
processed_count = 0
skipped_count = 0
duplicate_warnings = []  # เก็บรายการไฟล์ที่มีปัญหา
files_to_process = []    # เก็บไฟล์ที่พร้อมประมวลผล
files_to_skip = []       # เก็บไฟล์ที่ต้องข้าม

for image_path in image_files:
    filename = os.path.basename(image_path)
    #print(f"\n📸 ตรวจสอบรูป: {filename}")

    # ✅ ขั้นตอนที่ 1: ตรวจชื่อซ้ำ
    if filename in existing_filenames:
        print(f"🔍 พบชื่อซ้ำ: {filename}")

        # ✅ ขั้นตอนที่ 2: เช็ค image hash
        new_hash = get_image_hash(image_path)
        old_hash = existing_hashes.get(filename)

        if new_hash and old_hash and new_hash == old_hash:
            # ✅ รูปซ้ำเป๊ะ → ข้ามไฟล์
            print(f"⚠️ ข้ามรูปซ้ำเป๊ะ: {filename}")
            skipped_count += 1
            files_to_skip.append({
                'filename': filename,
                'path': image_path,
                'reason': 'รูปซ้ำเป๊ะ (เหมือนกันทุกประการ)',
                'action': 'ข้ามการประมวลผล - ไฟล์จะอยู่ที่เดิม'
            })
            continue
        else:
            # ✅ ชื่อซ้ำแต่รูปต่างกัน → เก็บไว้แจ้งเตือน
            duplicate_warnings.append({
                'filename': filename,
                'path': image_path,
                'reason': 'ชื่อไฟล์ซ้ำแต่เป็นรูปคนละอัน',
                'action': 'กรุณาเปลี่ยนชื่อไฟล์แล้วอัปโหลดใหม่'
            })
            print(f"⚠️ พบปัญหา: {filename} - ชื่อซ้ำแต่เป็นรูปต่างกัน")
            continue

    # ✅ ไฟล์ปกติ → เตรียมประมวลผล
    #print(f"✅ พร้อมประมวลผล: {filename}")
    files_to_process.append(image_path)

# ✅ แสดงสรุปการตรวจสอบและแจ้งเตือน
print(f"\n📊 สรุปการตรวจสอบไฟล์:")
print(f"✅ ไฟล์พร้อมประมวลผล: {len(files_to_process)} ไฟล์")
print(f"⚠️ ไฟล์ซ้ำที่ข้าม: {len(files_to_skip)} ไฟล์")
print(f"🚨 ไฟล์ที่ต้องแก้ไข: {len(duplicate_warnings)} ไฟล์")

# ✅ แจ้งเตือนรายการไฟล์ที่ต้องแก้ไข
if duplicate_warnings:
    warning_message = "พบไฟล์ที่มีชื่อซ้ำแต่เป็นรูปต่างกัน:\n\n"
    for i, item in enumerate(duplicate_warnings, 1):
        warning_message += f"{i}. {item['filename']}\n"
        warning_message += f"   ปัญหา: {item['reason']}\n"
        warning_message += f"   แก้ไข: {item['action']}\n\n"

    warning_message += "⚠️ ไฟล์เหล่านี้จะไม่ถูกประมวลผลจนกว่าจะแก้ไขชื่อไฟล์"

    show_alert_message("🚨 ต้องแก้ไขไฟล์ก่อนดำเนินการต่อ", warning_message, "error")

# ✅ แจ้งเตือนรายการไฟล์ที่ข้าม
if files_to_skip:
    skip_message = "พบรูปซ้ำที่จะข้ามการประมวลผล:\n\n"
    for i, item in enumerate(files_to_skip, 1):
        skip_message += f"{i}. {item['filename']}\n"
        skip_message += f"   สาเหตุ: {item['reason']}\n"
        skip_message += f"   การดำเนินการ: {item['action']}\n\n"

    skip_message += "ℹ️ ไฟล์เหล่านี้จะอยู่ในโฟลเดอร์เดิมและไม่ถูกย้าย"

    show_alert_message("ℹ️ รายการไฟล์ที่ข้ามการประมวลผล", skip_message, "info")

# ✅ ตรวจสอบว่ามีไฟล์ให้ประมวลผลหรือไม่
if not files_to_process:
    if duplicate_warnings:
        error_message = "ไม่มีไฟล์ใหม่ให้ประมวลผล\n\n"
        error_message += "กรุณาแก้ไขชื่อไฟล์ที่ซ้ำแล้วรันโค้ดใหม่อีกครั้ง"
        show_alert_message("🛑 หยุดการทำงาน", error_message, "error")
        sys.exit("หยุดการทำงานเนื่องจากไม่มีไฟล์ใหม่ให้ประมวลผล")
    else:
        info_message = "ไม่มีไฟล์ใหม่ให้ประมวลผล\n\n"
        info_message += "ไฟล์ทั้งหมดได้ถูกประมวลผลแล้ว"
        show_alert_message("ℹ️ การทำงานเสร็จสิ้น", info_message, "info")
        sys.exit("ไม่มีไฟล์ใหม่ให้ประมวลผล")

# ✅ ดำเนินการประมวลผลไฟล์ที่ผ่านการตรวจสอบ
#print(f"\n🚀 เริ่มประมวลผลไฟล์ {len(files_to_process)} ไฟล์...")

for image_path in files_to_process:
    filename = os.path.basename(image_path)
    #print(f"\n📸 ประมวลผล: {filename}")

    # เพิ่มข้อมูลเข้าเซตเพื่อตรวจสอบในรอบถัดไป
    img_hash = get_image_hash(image_path)
    if img_hash:
        existing_hashes[filename] = img_hash
        existing_filenames.add(filename)

    processed_count += 1

    # *** ใส่โค้ด OCR และการประมวลผลตรงนี้ ***
    # OCR และการประมวลผลปกติ...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 6.3 MB/s eta 0:00:00
🔍 โหลดข้อมูลไฟล์ที่เคยประมวลผล...
📋 พบไฟล์ที่เคยประมวลผล: 0 ไฟล์

📊 สรุปการตรวจสอบไฟล์:
✅ ไฟล์พร้อมประมวลผล: 1 ไฟล์
⚠️ ไฟล์ซ้ำที่ข้าม: 0 ไฟล์
🚨 ไฟล์ที่ต้องแก้ไข: 0 ไฟล์


In [4]:
# STEP 4: OCR ด้วย Google Cloud Vision API (แต่ยังไม่ลบไฟล์ pre)
from google.cloud import vision
import io
import os
import cv2
import json

# 🔧 ฟังก์ชันแก้ไขภาพให้เป็นขาวดำ + resize + save เป็นไฟล์ใหม่
def preprocess_image(input_path, output_path):
    image = cv2.imread(input_path, cv2.IMREAD_COLOR)
    if image is None:
        print(f"❌ ไม่สามารถอ่านภาพ: {input_path}")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    alpha = 1 # ความคมชัด
    beta = -11.5  # ความสว่าง
    adjusted = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)
    _, thresh = cv2.threshold(adjusted, 150, 255, cv2.THRESH_BINARY)
    resized = cv2.resize(thresh, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
    cv2.imwrite(output_path, resized)
    #print(f"✅ สร้างไฟล์ Preprocess แล้ว: {output_path}")
    return output_path

# 🔍 ฟังก์ชัน OCR ด้วย Google Cloud Vision API (พร้อม preprocess)
def ocr_google_vision_only(image_path):
    client = vision.ImageAnnotatorClient()
    file_root, file_ext = os.path.splitext(image_path)
    preprocessed_path = f"{file_root}_pre{file_ext}"

    preprocessed_path = preprocess_image(image_path, preprocessed_path)
    if not preprocessed_path or not os.path.exists(preprocessed_path):
        print(f"❌ ไม่พบไฟล์ preprocess: {preprocessed_path}")
        return "", None

    try:
        with io.open(preprocessed_path, 'rb') as image_file:
            content = image_file.read()
        image = vision.Image(content=content)
        response = client.text_detection(image=image)
        texts = response.text_annotations
        result = texts[0].description.strip() if texts else ""
        #print("\n📄 ผลลัพธ์ OCR:\n" + result)
    except Exception as e:
        print(f"❌ OCR ผิดพลาด: {e}")
        result = ""

    return result, preprocessed_path

# ✅ ใช้ OCR กับหลายภาพและเก็บ path และผลลัพธ์ไว้ใช้ต่อ
folder_path = os.path.join(PROJECT_DIR, config["FOLDER_PATH"]) #config.json
image_files = [os.path.join(folder_path, f)
               for f in os.listdir(folder_path)
               if f.lower().endswith((".jpg", ".jpeg", ".png"))]

print(f"📂 พบรูปภาพทั้งหมด {len(image_files)} ไฟล์")

ocr_results = []
preprocessed_files = []

for image_path in image_files:
    #print(f"📄 กำลังประมวลผล: {image_path}")
    ocr_result, pre_path = ocr_google_vision_only(image_path)
    if pre_path:
        preprocessed_files.append(pre_path)
    ocr_results.append({
        "image_path": image_path,
        "ocr_text": ocr_result
    })

# 💾 บันทึกไฟล์ไว้ใช้ใน STEP 5 และ STEP 5
with open("/content/preprocessed_files.json", "w") as f:
    json.dump(preprocessed_files, f)

with open("/content/ocr_results.json", "w") as f:
    json.dump(ocr_results, f)


📂 พบรูปภาพทั้งหมด 1 ไฟล์


In [5]:
# STEP 5: ลบไฟล์ชั่วคราว (.pre)

import os

def cleanup_preprocessed_image(preprocessed_path):
    try:
        if os.path.exists(preprocessed_path):
            os.remove(preprocessed_path)
            #print(f"🧹 ลบไฟล์ชั่วคราว: {os.path.basename(preprocessed_path)}")
        else:
            print(f"⚠️ ไม่พบไฟล์: {os.path.basename(preprocessed_path)}")
    except Exception as e:
        print(f"❌ ลบไฟล์ไม่ได้: {e}")

# 🔁 โหลดรายการไฟล์ที่ preprocess มาจาก STEP 4
import json
with open("/content/preprocessed_files.json", "r") as f:
    preprocessed_files = json.load(f)

for pre_file in preprocessed_files:
    cleanup_preprocessed_image(pre_file)


In [6]:
# ✅ ตั้งค่า Gemini
import google.generativeai as genai
import json
import os
import re
from datetime import datetime

genai.configure(api_key=config["API_KEY_GEMINI"])  #🔁 เปลี่ยนเป็นคีย์ของคุณ config.json
model = genai.GenerativeModel("gemini-1.5-flash")


In [7]:
# 🔧 ฟังก์ชันจัดการวันที่และเวลา

thai_months = {
    "ม.ค.": "ม.ค.", "ก.พ.": "ก.พ.", "มี.ค.": "มี.ค.", "เม.ย.": "เม.ย.",
    "พ.ค.": "พ.ค.", "มิ.ย.": "มิ.ย.", "ก.ค.": "ก.ค.", "ส.ค.": "ส.ค.",
    "ก.ย.": "ก.ย.", "ต.ค.": "ต.ค.", "พ.ย.": "พ.ย.", "ธ.ค.": "ธ.ค.",
    "ม.ค": "ม.ค.", "ก.พ": "ก.พ.", "มี.ค": "มี.ค.", "เม.ย": "เม.ย.",
    "พ.ค": "พ.ค.", "มิ.ย": "มิ.ย.", "ก.ค": "ก.ค.", "ส.ค": "ส.ค.",
    "ก.ย": "ก.ย.", "ต.ค": "ต.ค.", "พ.ย": "พ.ย.", "ธ.ค": "ธ.ค."
}

def format_thai_date(text):
    cleaned = re.sub(r"[,|\-]?\s*\d{1,2}[:.]\d{2}(:\d{2})?\s*น?\.?", "", text)
    match = re.search(r"(\d{1,2})\s+([^\s]+)\s+(\d{2,4})", cleaned)
    if not match:
        return ""

    day, month_raw, year_raw = match.group(1), match.group(2), match.group(3)
    month = thai_months.get(month_raw, month_raw)
    year = year_raw[-2:] if len(year_raw) == 4 else year_raw
    day = str(int(day))
    return f"{day} {month} {year}"

def format_time_24h(raw_time):
    raw_time = raw_time.strip().replace(".", ":")
    match = re.match(r"(\d{1,2}):(\d{2})", raw_time)
    if match:
        h, m = int(match.group(1)), int(match.group(2))
        if 0 <= h < 24 and 0 <= m < 60:
            return f"{h:02d}:{m:02d}"
    return raw_time

In [8]:
# 🔧 ฟังก์ชันแปลงชื่อธนาคารหรือผู้ให้บริการเป็นภาษาไทยเท่านั้น

bank_name_map = {
    # ธนาคาร
    "SCB": "ไทยพาณิชย์",
    "ไทยพาณิชย์": "ไทยพาณิชย์",
    "ธ.ไทยพาณิชย์": "ไทยพาณิชย์",
    "KBank": "กสิกรไทย",
    "กสิกรไทย": "กสิกรไทย",
    "KTB": "กรุงไทย",
    "กรุงไทย": "กรุงไทย",
    "BAY": "กรุงศรี",
    "กรุงศรี": "กรุงศรี",
    "Bangkok Bank": "กรุงเทพ",
    "ธนาคารกรุงเทพ": "กรุงเทพ",
    "TMB": "ทหารไทย",
    "CIMB": "ซีไอเอ็มบี",
    "UOB": "ยูโอบี",
    "ttb": "ทหารไทยธนชาต",
    "ธ.ทหารไทยธนชาต": "ทหารไทยธนชาต",
    "ธนาคารออมสิน": "ออมสิน",
}

def normalize_bank_name(name):
    if not name:
        return ""
    for key in bank_name_map:
        if key.lower() in name.lower():
            return bank_name_map[key]
    return ""  # ไม่ตรงกับรายชื่อที่กำหนด


In [9]:
# 🔁 Template สำหรับให้ Gemini แปลง OCR เป็น JSON

custom_categories = ["อาหาร", "เดินทาง", "ของใช้", "บันเทิง", "ที่พัก", "ค่าน้ำ", "ค่าไฟ", "บัตรเครดิต", "แอปสตรีมมิ่ง", "มือถือและอินเทอร์เน็ต", "สุขภาพ", "สัตว์เลี้ยง", "อื่นๆ"]

prompt_template = """
คุณคือระบบจัดหมวดหมู่ข้อมูลการโอนเงินจากข้อความ OCR
ให้แปลงข้อความด้านล่างให้อยู่ในรูปแบบ JSON ที่ valid 100%
(ใช้ double quote เท่านั้น และไม่มีข้อความอื่นนอกจาก JSON)
โดยมีหัวข้อ:
- วันที่
- เวลา
- ชื่อผู้โอน
- ธนาคารผู้โอน
- หมายเลขบัญชีผู้โอน
- ชื่อผู้รับโอน
- ธนาคารผู้รับ
- หมายเลขบัญชีผู้รับ
- เลขที่รายการ
- จำนวนเงิน
- ค่าธรรมเนียม
- หมวดหมู่ (เลือกจาก: {categories})

📌 หมายเหตุ:
หากพบคำว่า "AIS", "TRUE", "DTAC", "NT", "TOT", หรือข้อความที่เกี่ยวข้องกับผู้ให้บริการโทรศัพท์/อินเทอร์เน็ต ให้ใส่หมวดหมู่เป็น **"มือถือและอินเทอร์เน็ต"**
หากพบคำว่า "บ้าน", "สินเชื่อบ้าน", หรือข้อความที่เกี่ยวข้องกับบ้าน ให้ใส่หมวดหมู่เป็น **"ที่พัก"**

วันที่ที่ตรวจพบจากระบบ OCR: {detected_date}

OCR ข้อความ:
\"\"\"
{raw_text}
\"\"\"
"""


In [10]:
# STEP 5: ใช้ Gemini ในการ LLM
structured_results = {}

for entry in ocr_results:
    raw_text = entry["ocr_text"]
    img_path = entry["image_path"]
    img_name = os.path.basename(img_path)

    # 🔍 วิเคราะห์วันที่ด้วยฟังก์ชันของเราเอง (ไม่พึ่ง Gemini)
    formatted_date = format_thai_date(raw_text)

    # ✏️ สร้าง prompt ที่บอกวันที่ให้โมเดลชัดเจน
    prompt = prompt_template.format(
        raw_text=raw_text,
        detected_date=formatted_date,
        categories=", ".join(custom_categories)
    )

    # 🤖 เรียกใช้ Gemini
    response = model.generate_content(prompt)
    raw_output = response.text.strip()

    # 🧼 ลบ ```json ออกจากคำตอบของ Gemini (ถ้ามี)
    if raw_output.startswith("```json"):
        raw_output = raw_output[7:]
    if raw_output.startswith("```"):
        raw_output = raw_output[3:]
    if raw_output.endswith("```"):
        raw_output = raw_output[:-3]

    try:
        # 🧠 แปลง JSON และบังคับใช้วันที่/เวลา
        data = json.loads(raw_output)

        #data["วันที่"] = format_thai_date(raw_text)
        data["เวลา"] = format_time_24h(data.get("เวลา", ""))

        # ✅ แปลงชื่อธนาคารเป็นภาษาไทย
        data["ธนาคารผู้โอน"] = normalize_bank_name(data.get("ธนาคารผู้โอน", ""))
        data["ธนาคารผู้รับ"] = normalize_bank_name(data.get("ธนาคารผู้รับ", ""))

        structured_results[img_name] = data

    except Exception as e:
        print(f"\n❌ JSON decode failed for: {img_name}")
        print("❌ RAW Gemini Output:")
        print(response.text)


In [11]:
 # STEP 6: ใช้ Google Drive API สร้าง file_map (filename → file_id)
from googleapiclient.discovery import build
from google.oauth2 import service_account

SERVICE_ACCOUNT_PATH = os.path.join(PROJECT_DIR, "setting", config["SERVICE_ACCOUNT_PATH"]) #config.json
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

# ใช้ Service Account สร้าง Credentials
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_PATH, scopes=SCOPES)

# สร้าง Google Drive API client
drive_service = build('drive', 'v3', credentials=credentials)

FOLDER_ID = config["FOLDER_ID"]  # โฟลเดอร์ที่เก็บรูปใน Google Drive , config.json

# ดึงไฟล์ทั้งหมดในโฟลเดอร์
file_map = {}
page_token = None

while True:
    response = drive_service.files().list(
        q=f"'{FOLDER_ID}' in parents and trashed=false",
        spaces='drive',
        fields='nextPageToken, files(id, name)',
        pageToken=page_token
    ).execute()

    for file in response.get('files', []):
        file_map[file['name']] = file['id']

    page_token = response.get('nextPageToken', None)
    if page_token is None:
        break


In [12]:
# STEP 7: เขียนลง Google Sheet

import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

# สร้างตัวแปรกลางสำหรับเก็บหมวดหมู่ - ดึงค่ามาจาก custom_categories ที่มีอยู่แล้ว
try:
    CATEGORIES = custom_categories
    #print(f"✅ ใช้หมวดหมู่จาก custom_categories ({len(CATEGORIES)} รายการ)")
except NameError:
    # ถ้าไม่มี custom_categories ให้ใช้แค่ "อื่นๆ" เพียงหมวดเดียว
    CATEGORIES = ["อื่นๆ"]
    print(f"⚠️ ไม่พบ custom_categories ใช้หมวดหมู่เริ่มต้น: อื่นๆ")

# หรือเขียนแบบสั้นๆ:
# CATEGORIES = custom_categories if 'custom_categories' in globals() else ["อื่นๆ"]

# ฟังก์ชันแปลงวันที่ไทย dd/mm/yyyy (พ.ศ.) → yyyy-mm-dd (ค.ศ.)
def convert_thai_date_to_iso(thai_date: str) -> str:
    try:
        # ตรวจสอบว่าเป็นรูปแบบ dd/mm/yyyy หรือไม่
        if "/" not in thai_date:
            return thai_date

        parts = thai_date.split("/")
        if len(parts) != 3:
            return thai_date

        day, month, year = parts

        # แปลง พ.ศ. เป็น ค.ศ.
        year_ad = int(year) - 543

        # ตรวจสอบความถูกต้องของวันที่
        try:
            datetime(year_ad, int(month), int(day))
        except ValueError:
            return thai_date

        # คืนค่าในรูปแบบ yyyy-mm-dd
        return f"{year_ad}-{month.zfill(2)}-{day.zfill(2)}"

    except Exception as e:
        print(f"Error converting date {thai_date}: {e}")
        return thai_date

# วิธีที่ 2: บังคับให้ Google Sheets ตีความเป็นวันที่
def create_date_formula(thai_date: str) -> str:
    try:
        # แปลงเดือนภาษาไทยเป็นตัวเลข
        thai_months = {
            'ม.ค.': 1, 'มกรา': 1, 'มกราคม': 1,
            'ก.พ.': 2, 'กุมภา': 2, 'กุมภาพันธ์': 2,
            'มี.ค.': 3, 'มีนา': 3, 'มีนาคม': 3,
            'เม.ย.': 4, 'เมษา': 4, 'เมษายน': 4,
            'พ.ค.': 5, 'พฤษภา': 5, 'พฤษภาคม': 5,
            'มิ.ย.': 6, 'มิถุนา': 6, 'มิถุนายน': 6,
            'ก.ค.': 7, 'กรกฎา': 7, 'กรกฎาคม': 7,
            'ส.ค.': 8, 'สิงหา': 8, 'สิงหาคม': 8,
            'ก.ย.': 9, 'กันยา': 9, 'กันยายน': 9,
            'ต.ค.': 10, 'ตุลา': 10, 'ตุลาคม': 10,
            'พ.ย.': 11, 'พฤศจิกา': 11, 'พฤศจิกายน': 11,
            'ธ.ค.': 12, 'ธันวา': 12, 'ธันวาคม': 12
        }

        # รองรับเฉพาะรูปแบบ "dd เดือน yy" เช่น "22 ม.ค. 64"
        parts = thai_date.split()
        if len(parts) == 3:
            day_str, month_str, year_str = parts

            # หาเดือนจาก dictionary
            month_num = None
            for thai_month, num in thai_months.items():
                if thai_month in month_str:
                    month_num = num
                    break

            if month_num:
                day = int(day_str)
                year = int(year_str)

                # แปลงปี 2 หลักเป็นปี พ.ศ. แล้วแปลงเป็น ค.ศ.
                if year < 100:
                    year_be = 2500 + year  # เช่น 64 -> 2564 (พ.ศ.)
                    year_ad = year_be - 543  # แปลงเป็น ค.ศ. เช่น 2564 -> 2021
                else:
                    year_ad = year - 543 if year > 2400 else year

                # ตรวจสอบความถูกต้องของวันที่
                try:
                    datetime(year_ad, month_num, day)
                    #print(f"แปลงวันที่: {thai_date} -> {day:02d}/{month_num:02d}/{year_be} (พ.ศ.)")

                    # ใช้ปี พ.ศ. ในชีทเพื่อให้ตรงกับใบเสร็จ
                    return f"{day:02d}/{month_num:02d}/{year_be}"

                except ValueError:
                    print(f"วันที่ไม่ถูกต้อง: {thai_date}")
                    return thai_date
            else:
                print(f"หาเดือนไม่เจอ: {month_str}")
                return thai_date
        else:
            print(f"รูปแบบวันที่ไม่ถูกต้อง: {thai_date} (ต้องเป็น 'วัน เดือน ปี')")
            return thai_date

    except Exception as e:
        #print(f"Error creating date formula {thai_date}: {e}")
        return thai_date

# ฟังก์ชันสำหรับตั้งค่า dropdown หมวดหมู่ (แก้ไขแล้ว)
def setup_category_dropdown(sheet, start_row, end_row):
    """
    ตั้งค่า dropdown สำหรับหมวดหมู่ในคอลัมน์ L (หมวดหมู่) - เปลี่ยนจาก M เป็น L
    """
    # ใช้หมวดหมู่ที่กำหนดไว้แล้วด้านบน
    categories = CATEGORIES

    try:
        # ตั้งค่า Data Validation สำหรับคอลัมน์ L (หมวดหมู่)
        validation_rule = {
            "condition": {
                "type": "ONE_OF_LIST",
                "values": [{"userEnteredValue": category} for category in categories]
            },
            "showCustomUi": True,
            "strict": True
        }

        # ประยุกต์ใช้กับช่วง L2:L (จากแถว 2 ลงไปทั้งหมด) - เปลี่ยนจาก M เป็น L
        range_name = f'L{start_row}:L{end_row}' if end_row else f'L{start_row}:L'

        # ใช้ batch_update สำหรับการตั้งค่า validation
        sheet.spreadsheet.batch_update({
            "requests": [
                {
                    "setDataValidation": {
                        "range": {
                            "sheetId": sheet.id,
                            "startRowIndex": start_row - 1,  # ลบ 1 เพราะ API ใช้ 0-based index
                            "endRowIndex": end_row if end_row else 1000,  # กำหนดแถวสุดท้าย หรือ 1000 ถ้าไม่กำหนด
                            "startColumnIndex": 11,  # คอลัมน์ L (0-based คือ 11) - เปลี่ยนจาก 12 เป็น 11
                            "endColumnIndex": 12     # ถึงคอลัมน์ L - เปลี่ยนจาก 13 เป็น 12
                        },
                        "rule": validation_rule
                    }
                }
            ]
        })

        #print(f"✅ ตั้งค่า dropdown หมวดหมู่สำเร็จ (ช่วง {range_name})")
        #print(f"📋 มีหมวดหมู่ทั้งหมด {len(categories)} รายการ: {', '.join(categories)}")

    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดในการตั้งค่า dropdown: {e}")


# 1. กำหนด Scope ที่จำเป็น
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# 2. ชี้ไปยัง Service Account JSON
SERVICE_ACCOUNT_FILE = os.path.join(PROJECT_DIR, "setting", config["SERVICE_ACCOUNT_FILE"]) #config.json
creds = ServiceAccountCredentials.from_json_keyfile_name(
    SERVICE_ACCOUNT_FILE,
    scopes
)

# 3. สร้าง gspread client
gc = gspread.authorize(creds)

# 4. เปิด Spreadsheet
SPREADSHEET_NAME = config["SPREADSHEET_NAME"]  #config.json
sheet = gc.open(SPREADSHEET_NAME).sheet1

# 5. เคลียร์และตั้งหัวตาราง
header = [
    "ไฟล์ภาพ", "วันที่", "เวลา", "ชื่อผู้โอน", "ธนาคารผู้โอน", "หมายเลขบัญชีผู้โอน",
    "ชื่อผู้รับโอน", "ธนาคารผู้รับ", "หมายเลขบัญชีผู้รับ", #"เลขที่รายการ",
    "จำนวนเงิน", "ค่าธรรมเนียม", "หมวดหมู่"
]
sheet.clear()
sheet.append_row(header)

# เก็บตำแหน่งแถวเริ่มต้นและสุดท้าย สำหรับตั้งค่า dropdown
start_row = 2  # เริ่มจากแถวที่ 2 (หลังจาก header)
data_rows = []

# 6. เขียนข้อมูลจาก structured_results
for filename, data in structured_results.items():
    try:
        # รองรับ dict หรือ json string
        if isinstance(data, str):
            data = json.loads(data)

        # แปลงวันที่โดยใช้ DATE formula ของ Google Sheets
        original_date = data.get("วันที่", "")
        date_formula = create_date_formula(original_date)

        # แสดงข้อมูลการแปลงวันที่
        #print(f"ไฟล์: {filename}")
        #print(f"วันที่ต้นฉบับ: {original_date}")
        #print(f"วันที่ที่แปลงเป็น formula: {date_formula}")

        # ลิงก์ไฟล์ภาพ Google Drive
        file_id = file_map.get(filename, "")
        file_link_formula = f'=HYPERLINK("https://drive.google.com/file/d/{file_id}/view?usp=sharing", "{filename}")' if file_id else filename

        row = [
            file_link_formula,
            date_formula,
            data.get("เวลา", ""),
            data.get("ชื่อผู้โอน", ""),
            data.get("ธนาคารผู้โอน", ""),
            data.get("หมายเลขบัญชีผู้โอน", ""),
            data.get("ชื่อผู้รับโอน", ""),
            data.get("ธนาคารผู้รับ", ""),
            data.get("หมายเลขบัญชีผู้รับ", ""),
            #data.get("เลขที่รายการ", ""),
            data.get("จำนวนเงิน", ""),
            data.get("ค่าธรรมเนียม", ""),
            data.get("หมวดหมู่", ""),  # จะเป็นค่าว่างเพื่อให้ผู้ใช้เลือกจาก dropdown
        ]

        sheet.append_row(row, value_input_option='USER_ENTERED')
        data_rows.append(len(sheet.get_all_values()))

        # ตั้งค่า format วันที่สำหรับคอลัมน์ B (วันที่)
        current_row = len(sheet.get_all_values())
        sheet.format(f"B{current_row}", {
            "numberFormat": {
                "type": "DATE",
                "pattern": "dd-mm-yyyy"
            }
        })

        #print(f"✅ เขียนข้อมูลไฟล์ {filename} ลงชีทสำเร็จ (แถว {current_row})\n")

    except Exception as e:
        print(f"❌ Error writing {filename}: {e}")

# 7. ตั้งค่า dropdown สำหรับหมวดหมู่
if data_rows:
    end_row = max(data_rows)
    setup_category_dropdown(sheet, start_row, end_row)
else:
    # ถ้าไม่มีข้อมูล ให้ตั้งค่า dropdown สำหรับแถว 2-100
    setup_category_dropdown(sheet, start_row, 100)

#print("🎉 อัปเดตข้อมูลลง Google Sheets เสร็จสิ้น!")






SpreadsheetNotFound: <Response [200]>

In [ ]:
# ✅ STEP 8: ย้ายเฉพาะรูปที่ประมวลผลแล้วไปโฟลเดอร์ processed
print(f"\n📂 ย้ายไฟล์ที่ประมวลผลแล้ว...")
moved_count = 0

for img_path in files_to_process:
    try:
        dest_path = os.path.join(PROCESSED_FOLDER, os.path.basename(img_path))
        os.rename(img_path, dest_path)
        moved_count += 1
        #print(f"📁 ย้ายแล้ว: {os.path.basename(img_path)}")
    except Exception as e:
        print(f"❌ ไม่สามารถย้ายไฟล์ {os.path.basename(img_path)}: {e}")

#print(f"✅ ย้ายไฟล์สำเร็จ: {moved_count} ไฟล์")

# ✅ STEP 9: ลบข้อมูลในชีทหากไฟล์ต้นฉบับถูกลบไปแล้ว
existing_images = set(os.listdir(INPUT_FOLDER)) | set(os.listdir(PROCESSED_FOLDER))
existing_data = sheet.get_all_values()
existing_filenames_in_sheet = set()
deleted_count = 0

# ดึงชื่อจากสูตร HYPERLINK ในคอลัมน์ A
for i in range(len(existing_data) - 1, 0, -1):  # ลบจากล่างขึ้นบน
    row = existing_data[i]
    if row and 'HYPERLINK' in row[0]:
        name = row[0].split('"')[-2]
        if name not in existing_images:
            sheet.delete_rows(i + 1)  # +1 เพราะ Google Sheet นับจาก 1
            deleted_count += 1
        else:
            existing_filenames_in_sheet.add(name)

print(f"🗑️ ลบข้อมูล {deleted_count} รายการ เพราะไม่มีภาพต้นฉบับแล้ว")

# ✅ STEP 10: เพิ่มเฉพาะข้อมูลใหม่ลงชีท
new_entries_count = 0
for image_path in files_to_process:
    filename = os.path.basename(image_path)

    if filename not in existing_filenames_in_sheet:
        # เตรียมลิงก์ Google Drive
        file_id = file_map.get(filename, "")
        file_link_formula = f'=HYPERLINK("https://drive.google.com/file/d/{file_id}/view?usp=sharing", "{filename}")' if file_id else filename

        # ดึงข้อมูลจาก structured_results
        data = structured_results.get(filename)
        if isinstance(data, str):
            data = json.loads(data)

        date_formula = create_date_formula(data.get("วันที่", ""))

        row = [
            file_link_formula,
            date_formula,
            data.get("เวลา", ""),
            data.get("ชื่อผู้โอน", ""),
            data.get("ธนาคารผู้โอน", ""),
            data.get("หมายเลขบัญชีผู้โอน", ""),
            data.get("ชื่อผู้รับโอน", ""),
            data.get("ธนาคารผู้รับ", ""),
            data.get("หมายเลขบัญชีผู้รับ", ""),
            data.get("จำนวนเงิน", ""),
            data.get("ค่าธรรมเนียม", ""),
            data.get("หมวดหมู่", ""),
        ]

        sheet.append_row(row, value_input_option='USER_ENTERED')
        new_entries_count += 1
        print(f"📝 เพิ่มข้อมูลใหม่: {filename}")

print(f"✅ เพิ่มข้อมูลใหม่: {new_entries_count} รายการ")

# ✅ สรุปการทำงานทั้งหมด
success_message = f"""การประมวลผลเสร็จสิ้น!

📊 สถิติการทำงาน:
✅ ประมวลผลสำเร็จ: {processed_count} ไฟล์
📁 ย้ายไฟล์แล้ว: {moved_count} ไฟล์
⚠️ ข้ามรูปซ้ำ: {skipped_count} ไฟล์
🚨 ไฟล์ที่ต้องแก้ไข: {len(duplicate_warnings)} ไฟล์

📂 ไฟล์ที่ข้ามจะอยู่ในโฟลเดอร์เดิม
📊 ข้อมูลใน Google Sheets ได้รับการอัปเดตแล้ว"""

show_alert_message("🎉 การทำงานเสร็จสิ้น", success_message, "info")

print("🎉 อัปเดตข้อมูลลง Google Sheets เสร็จสิ้น!")